[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Image Processing with Python

## Quantization & Dithering

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 04/10/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/ImageProcessingPython/0002SciKitImageBasics.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

from numba import njit, vectorize

# Image Processing
import skimage as ski

# Machine Learning


# Miscellaneous
import os
from platform import python_version
import random
import timeit

# Typing
from typing import Callable, List, Optional, Tuple

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatRangeSlider, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants



In [ ]:
# Fixel Algorithms Packages


In [ ]:
# General Auxiliary Functions



## Image Quantization & Dithering

Quantization is the process of _reducing_ / _limiting_ the number of unique values within a dynamic range.  
Dithering is trying to mimic a larger amount of values by adding quantized noise spatially.

This notebooks implements some approaches of each method. 

* <font color='brown'>(**#**)</font> In some sense _Qunatization_ is equivalent to _Sampling_ on the value domain.
* <font color='brown'>(**#**)</font> Both concepts are commonly used in the _Signal Processing_ context.

In [ ]:
# Parameters

# Image by [AeroDraws](https://www.deviantart.com/aerodraws/gallery)
# imgUrl = 'https://i.imgur.com/SCgj4Nh.png' #<! https://www.deviantart.com/aerodraws/art/Grayscale-Portrait-Study-847900592
imgUrl = 'https://i.postimg.cc/Kj7CBXTn/Img.png' #<! https://www.deviantart.com/aerodraws/art/Grayscale-Portrait-Study-847900592

numLevels = 4 #<! Quantization levels (L)
lNumLevels = [32, 16, 8, 4, 2]
numSamples = 1_000 #<! Samples for an evaluation

## Generate Data

This section loads the image used to evaluate the results.  

In [ ]:
# Load / Generate Data

mI = ski.io.imread(imgUrl)
mI = ski.util.img_as_float64(mI)
mI = np.mean(mI, axis = 2)

numLevelsImg = len(np.unique(mI)) #<! Number of unique levels in the image


In [ ]:
# Display the Data

hF, hA = plt.subplots(figsize = (8, 6))
hA.imshow(mI, cmap = 'gray')
hA.set_title(f'Input Image, Number of Levels: {numLevelsImg}');

## Quantization

Quantization of $L$ levels can be broken into 2 main steps:

 - Partitioning  
   Setting $L + 1$ edges splitting the entire dynamic range into $L$ segments.
 - Mapping  
   Mapping each segment into a valid value.

This section implements a _Quantizer_.

* <font color='brown'>(**#**)</font> Some use $L - 1$ edges where the 2 implicit additional edges are the extreme values of the _Dynamic Range_.


### Quantizer

The implemented _Quantizer_ partitions the range uniformly within $\left[ {f}_{min} - \frac{q}{2}, {f}_{max} + \frac{q}{2} \right]$ where $q$ is the segment length.  
The _Quantizer_ parameters are `valMin`, `valMax` and `numLevels`. The input value is given by `valIn`.  
The _Quantizer_ should be implemented scalar wise.

In [ ]:
# Qunatizer

#===========================Fill This===========================#
# 1. Implement the Quantizer as a function.
# 2. The function parameters are `valIn`, `valMin`, `valMax` and `numLevels`.
# 3. The function should be written for scalar input in mind.
# !! Numpy will be used to vectorize the function.

def Quantizer( valIn: float, numLevels: int, valMin: float, valMax: float ) -> float:    
    """
    Quantizes a given input value into one of the specified discrete levels within a given range.

    This function maps a continuous input value (`valIn`) within the range `[valMin, valMax]` 
    to one of `numLevels` equally spaced quantization levels in that range. The output value is 
    calculated to be the nearest quantized level based on the input.

    Parameters
    ----------
    valIn : float
        The input value to be quantized, expected to be in the range `[valMin, valMax]`.
    numLevels : int
        The number of quantization levels. Must be an integer greater than 1.
    valMin : float
        The minimum value of the quantization range.
    valMax : float
        The maximum value of the quantization range.

    Returns
    -------
    valOut : float
        The quantized output value, mapped to one of the `numLevels` discrete levels within the 
        range `[valMin, valMax]`.

    Notes
    -----
    - This function assumes that `valIn` is within the specified range `[valMin, valMax]`.
      If `valIn` falls outside this range, the result might not be as expected.
    - The quantization levels are linearly spaced between `valMin` and `valMax`, with each level 
      representing a segment of the input range.

    Examples
    --------
    >>> Quantizer(0.3, 4, 0.0, 1.0)
    0.3333333333333333  # Maps 0.3 to one of the 4 quantization levels within [0.0, 1.0]

    >>> Quantizer(0.75, 8, 0.0, 1.0)
    0.7142857142857143  # Maps 0.75 to one of the 8 quantization levels within [0.0, 1.0]

    Calculation Details
    -------------------
    The quantization grid is defined by evenly spaced levels between `valMin` and `valMax`. 
    For example, with `numLevels = 4`, `valMin = 0.0`, and `valMax = 1.0`, the quantization 
    levels will correspond to values approximately equal to `[0.0, 0.333, 0.667, 1.0]`.
    """

    # For `numLevels` = 4 the partition grid should be  [-0.16666667,  0.16666667,  0.5       ,  0.83333333,  1.16666667]
    # binSize = 1 / (numLevels - 1)
    # vQ = np.linspace(0 - binSize / 2, 1 + binSize / 2, numLevels + 1)[1:-1] #<! Quantization Grid
    # valOut = np.digitize(valIn, vQ) / (numLevels - 1)
    valOut = np.round(((valIn - valMin) / (valMax - valMin)) * (numLevels - 1)) / (numLevels - 1)

    return valOut

#===============================================================#

In [ ]:
# NumPy Vectorization

# Vectorize the function for the input yet not the parameters
hQuantizer = np.vectorize(Quantizer, excluded = ['numLevels', 'valMin', 'valMax'])

In [ ]:
# Apply Function for a Line

vI = np.linspace(0, 1, numSamples)
vF = hQuantizer(vI, numLevels, np.min(vI), np.max(vI)) #<! Using the vectorization

In [ ]:
# Display Results

hF, hA = plt.subplots(nrows = 1, ncols = 1, figsize = (7, 5))

hA.plot(vI, vI, ls = ':', lw = 2, label = 'Input Signal')
hA.plot(vI, vF, lw = 2, label = 'Quantized Signal')
hA.grid(True)
hA.set_xlabel('Input Value')
hA.set_ylabel('Output Value')
hA.set_title(f'Quantizing on the Range [0, 1] with {numLevels} Levels')
hA.set_title(f'Quantizing on the Range [0, 1] with {numLevels} Levels')

hA.legend();

* <font color='red'>(**?**)</font> Is this a _Uniform Quantization_?

In [ ]:
# Quantize the Image

numImg = len(lNumLevels) + 1

hF, vHa = plt.subplots(nrows = 1, ncols = numImg, figsize = (numImg * 4 + 1, 4))
vHa = vHa.flat

for ii, hA in enumerate(vHa):
    if (ii == 0):
        hA.imshow(mI, cmap = 'gray', vmin = 0, vmax = 1)
        hA.set_title('Input Image')
    else:
        paramL = lNumLevels[ii - 1]
        hA.imshow(hQuantizer(mI, paramL, 0.0, 1.0), cmap = 'gray', vmin = 0, vmax = 1)
        hA.set_title(f'Quantized Image, L = {paramL}')
    
    hA.set_xticks([])
    hA.set_yticks([])


## Image Dithering

The _Dithering_ operation tries to mitigate some of the artifacts generated by the _Quantization_ process.

This section implement a Floyd Steinberg Dithering style method.

* <font color='brown'>(**#**)</font> The same framework can serve for [Floyd Steinberg Dithering](https://en.wikipedia.org/wiki/Floyd%E2%80%93Steinberg_dithering) and [Atkinson Dithering](https://en.wikipedia.org/wiki/Atkinson_dithering).

### Floyd Steinberg Style Dithering

This section implements a Floyd Steinberg like dithering.  
The concept is defining a set of pixels which are not processed in the current location of the scan path.  
Then push a weighted error to them.

In [ ]:
# Dithering

#===========================Fill This===========================#
# 1. Implement the Dithering as a function.
# !! Wikipedia may be used as reference.

def DitherImage( mI: np.ndarray, numLevels: int, vW: np.ndarray ) -> np.ndarray:    
    """
    Applies quantization with error diffusion (dithering) to an image.

    This function performs quantization on a grayscale image with a specified number of levels, using dithering to distribute the quantization error to neighboring pixels.
    The dithering is implemented by adjusting nearby pixels based on the error from quantizing the current pixel, which can help produce visually smoother results and reduce banding artifacts in the quantized image.

    Parameters
    ----------
    mI : np.ndarray
        Input grayscale image as a 2D NumPy array with pixel values normalized in the range [0, 1].
    numLevels : int
        Number of quantization levels. Must be an integer greater than 1.
    lW : np.ndarray
        Vector of weights for each position in `lPos`, used to control how much of the quantization error is propagated to each neighboring pixel.
        The weights will be normalized to ensure they sum to 1.

    Returns
    -------
    mO : ndarray
        Quantized image as a 2D NumPy array with the same shape as the input `mI`. The pixel values are integers from 0 to `numLevels - 1`, corresponding to the quantized levels.

    Notes
    -----
    - The shifts of the pixels to propagate the error are right and bottom. See `lPos`.
    - This function assumes that the input image `mI` is normalized to the range [0, 1]. If the input image has a different range, it should be normalized before calling this function.
    - Error diffusion is applied based on the offsets and weights provided in `lPos` and `vW`, respectively.
    - The function uses Floyd Steinberg dithering or similar techniques by propagating the quantization error to neighboring pixels, which can help create the appearance of intermediate tones.

    References
    ----------
    This function is based on Floyd-Steinberg dithering. See more about dithering and error diffusion at:
    https://web.archive.org/web/20110410051449/http://www.efg2.com/Lab/Library/ImageProcessing/DHALF.TXT

    Example
    -------
    >>> import numpy as np
    >>> mI = np.random.rand(5, 5)  #<! Random 5x5 grayscale image
    >>> numLevels = 4  # Quantize to 4 levels
    >>> vW = np.array([7, 3, 5, 1])  #<! Floyd Steinberg weights
    >>> mO = DitherImage(image, numLevels, vW)
    """
    
    lPos = [(0, 1), (1, -1), (1, 0), (1, 1)]  #<! Floyd Steinberg offsets
    
    if (numLevels < 1) or (numLevels != round(numLevels)):
        raise ValueError(f'The `numLevels` must be a positive integer')
    
    vW = vW / np.sum(vW) #<! Normalize weights
    
    numRows = np.size(mI, 0)
    numCols = np.size(mI, 1)
    
    mI = np.copy(mI)
    mO = np.zeros_like(mI)
    for ii in range(numRows):
        for jj in range(numCols):
            # Quantize: Partition & Map
            mO[ii, jj] = Quantizer(mI[ii, jj], numLevels, 0.0, 1.0)
            # Error
            valE = mI[ii, jj] - mO[ii, jj]
            # Propagate the error (Use `lPos` shifts and `vW` weights)
            for (mm, nn), valW in zip(lPos, vW):
                mm += ii
                nn += jj
                if (mm < numRows) and (nn < numCols):
                    mI[mm, nn] += valE * valW
    
    return mO

#===============================================================#

In [ ]:
# Generate the Weights Vector

vW = np.array([7, 3, 5, 1])

* <font color='blue'>(**!**)</font> Replace the weights with [Atkinson Dithering](https://en.wikipedia.org/wiki/Atkinson_dithering) weights.

In [ ]:
# Dither the Image

numImg = len(lNumLevels) + 1

hF, vHa = plt.subplots(nrows = 1, ncols = numImg, figsize = (numImg * 4 + 1, 4))
vHa = vHa.flat

for ii, hA in enumerate(vHa):
    if (ii == 0):
        hA.imshow(mI, cmap = 'gray', vmin = 0, vmax = 1)
        hA.set_title('Input Image')
    else:
        paramL = lNumLevels[ii - 1]
        hA.imshow(DitherImage(mI, paramL, vW), cmap = 'gray', vmin = 0, vmax = 1, resample = False) #<! Disable interpolation to force values
        hA.set_title(f'Dithered Image, L = {paramL}')
    
    hA.set_xticks([])
    hA.set_yticks([])
